In [5]:
pip install faker

   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
    --------------------------------------- 0.0/2.0 MB 1.3 MB/s eta 0:00:02
   - -------------------------------------- 0.1/2.0 MB 907.3 kB/s eta 0:00:03
   -- ------------------------------------- 0.1/2.0 MB 930.9 kB/s eta 0:00:03
   ---- ----------------------------------- 0.2/2.0 MB 1.5 MB/s eta 0:00:02
   ------ --------------------------------- 0.3/2.0 MB 1.6 MB/s eta 0:00:02
   ---------- ----------------------------- 0.5/2.0 MB 2.0 MB/s eta 0:00:01
   ------------- -------------------------- 0.7/2.0 MB 2.3 MB/s eta 0:00:01
   ---------------- ----------------------- 0.8/2.0 MB 2.5 MB/s eta 0:00:01
   ------------------- -------------------- 1.0/2.0 MB 2.6 MB/s eta 0:00:01
   ----------------------- ---------------- 1.1/2.0 MB 2.8 MB/s eta 0:00:01
   -------------------------- ------------- 1.3/2.0 MB 2.8 MB/s eta 0:00:01
   ----------------------------- ---------- 1.5/2.0 MB 3.0 MB/s eta 0:00:01
   ------------

In [26]:
# reocdb_faker_sql_compatible.py
import os, random, csv
from datetime import timedelta
import pandas as pd
import numpy as np
from faker import Faker

# =========================
# CONFIG
# =========================
SEED = 42
faker = Faker()
random.seed(SEED); np.random.seed(SEED); Faker.seed(SEED)

NUM_SEGMENTS        = 60
NUM_EMPLOYEES       = 400
NUM_CUSTOMERS       = 8000
NUM_PROJECTS        = 180
NUM_CONTRACTORS     = 300
NUM_UNITS           = 9000
NUM_TRANSACTIONS    = 7000
NUM_PAYMENT_METHODS = 600
NUM_OWNERS          = 4500
NUM_INVESTORS       = 600
NUM_PORTFOLIOS      = 400

OUTPUT_PATH = "reocdb_faker"
os.makedirs(OUTPUT_PATH, exist_ok=True)

# =========================
# HELPERS
# =========================
segment_names     = ["Premium","Budget","Investors","Renters","Corporate","Retail"]
employee_roles    = ["Sales","Manager","Agent","Account","Project Manager","Support"]
customer_types    = ["Buyer","Renter","Investor","Prospect"]
unit_types        = ["Apartment","Villa","Studio","Commercial"]
unit_statuses     = ["Available","Sold","Reserved","Under Construction"]
tx_payment_status = ["Paid","Pending","Cancelled"]
transaction_types = ["Sale","Rent","Deposit","Refund"]
portfolio_labels  = ["Growth","Income","Balanced","Speculative","LongTerm"]

def unique_value(gen_func, used_set, max_tries=2000):
    for _ in range(max_tries):
        v = gen_func()
        if v and v not in used_set:
            used_set.add(v); return v
    base = gen_func(); i = 1
    while f"{base}-{i}" in used_set: i += 1
    used_set.add(f"{base}-{i}"); return f"{base}-{i}"

def choose_or_none(options, none_weight=0.0):
    if none_weight <= 0: return random.choice(options)
    pool = [None] * int(none_weight * 100) + list(options)
    return random.choice(pool)

def clean_text_cols(df):
    for col in df.select_dtypes(include=["object"]).columns:
        df[col] = (df[col].astype(str)
                         .str.replace(r"[\r\n]+", " ", regex=True)
                         .str.strip())
    return df

used_emails, used_phones = set(), set()
def gen_unique_email(company=False):
    return unique_value(lambda: faker.company_email() if company else faker.email(), used_emails)
def gen_unique_phone():
    return unique_value(lambda: faker.phone_number(), used_phones)

# =========================
# 1) Segment
# =========================
df_Segment = pd.DataFrame([{
    "segment_id": i,
    "segment_name": segment_names[(i-1) % len(segment_names)],
    "description": faker.sentence(nb_words=6),
    "target_income_range": f"{random.randint(10000,40000)}-{random.randint(50000,150000)}"
} for i in range(1, NUM_SEGMENTS+1)])

# =========================
# 2) Portfolio
# =========================
used_portfolio_names = set()
portfolios = []
for i in range(1, NUM_PORTFOLIOS+1):
    pname = unique_value(lambda: random.choice(portfolio_labels) + " " + faker.lexify(text="??").upper(),
                         used_portfolio_names)
    portfolios.append({
        "portfolio_id": i,
        "portfolio_name": pname,
        "description": faker.sentence(nb_words=8),
        "Due_Date": faker.date_between(start_date="-2y", end_date="+2y"),
        "escrow": round(np.random.uniform(10_000, 500_000), 2),
        "estate_cost": round(np.random.uniform(100_000, 5_000_000), 2),
    })
df_Portfolio = pd.DataFrame(portfolios)

# =========================
# 3) Payment_Method
# =========================
df_Payment_Method = pd.DataFrame([{
    "payment_method_number": i,
    "type": random.choice(["Bank Transfer","Credit Card","Cash","Cheque","Online"])
} for i in range(1, NUM_PAYMENT_METHODS+1)])

# =========================
# 4) Project
# =========================
projects = []
for i in range(1, NUM_PROJECTS+1):
    start = faker.date_between(start_date="-5y", end_date="-1y")
    end = start + timedelta(days=365 * random.randint(1, 4))
    projects.append({
        "project_id": i,
        "project_name": f"{faker.city()} Project {i}",
        "start_date": start,
        "end_date": end,
        "location": faker.city(),
        "unit_id": None   # FK to Unit, will set after Unit generated
    })
df_Project = pd.DataFrame(projects)

# =========================
# 5) Employee
# =========================
df_Employee = pd.DataFrame([{
    "Employee_id": i,
    "Employee_name": faker.name(),
    "role": random.choice(employee_roles)[:20],
    "phone": gen_unique_phone(),
    "email": gen_unique_email(company=True),
    "project_id": choose_or_none(list(range(1, NUM_PROJECTS+1)), 0.8)
} for i in range(1, NUM_EMPLOYEES+1)])

# =========================
# 6) Customer
# =========================
df_Customer = pd.DataFrame([{
    "customer_id": i,
    "first_name": faker.first_name(),
    "last_name": faker.last_name(),
    "gender": random.choice(["Male","Female","Other"]),
    "age": random.randint(18, 85),
    "phone": gen_unique_phone(),  # UNIQUE
    "email": gen_unique_email(),
    "avg_income": round(np.random.uniform(5_000, 200_000), 2),
    "customer_type": random.choice(customer_types)[:30],
    "is_ReferenceOnly": random.choice(["T","F"]) if random.random()<0.05 else "F",
    "segment_id": random.randint(1, NUM_SEGMENTS),
    "Employee_id": choose_or_none(list(range(1, NUM_EMPLOYEES+1)), 0.75)
} for i in range(1, NUM_CUSTOMERS+1)])

# =========================
# 7) Transactions
# =========================
transactions = []
for i in range(1, NUM_TRANSACTIONS+1):
    amount = round(np.random.uniform(2_000, 5_000_000), 2)
    transactions.append({
        "transaction_id": i,
        "name": f"TXN-{i}-{faker.lexify(text='???').upper()}",
        "transaction_date": faker.date_between(start_date="-4y", end_date="today"),
        "amount": amount,
        "tax": round(amount * random.choice([0.0, 0.05, 0.1, 0.14]), 2),
        "discount_amount": round(amount * random.choice([0.0, 0.01, 0.03, 0.05]), 2),
        "payment_status": random.choice(tx_payment_status),
        "transaction_type": random.choice(transaction_types),
        "customer_id": random.randint(1, NUM_CUSTOMERS),
        "payment_method_number": random.randint(1, NUM_PAYMENT_METHODS),
        "employee_id": choose_or_none(list(range(1, NUM_EMPLOYEES+1)), 0.8)
    })
df_Transactions = pd.DataFrame(transactions)
txn_by_id = df_Transactions.set_index("transaction_id")

# =========================
# 8) Unit
# =========================
units = []
for i in range(1, NUM_UNITS+1):
    txn_fk = None; cust_fk = None
    if random.random() < 0.35:
        # Only link to Sale/Rent
        while True:
            cand = random.randint(1, NUM_TRANSACTIONS)
            if txn_by_id.loc[cand, "transaction_type"] in {"Sale","Rent"}:
                txn_fk = cand
                cust_fk = int(txn_by_id.loc[cand, "customer_id"])
                break
    units.append({
        "unit_id": i,
        "type": random.choice(unit_types),
        "area": round(np.random.uniform(35.0, 450.0), 2),
        "status": random.choice(unit_statuses),
        "price": round(np.random.uniform(50_000, 2_500_000), 2),
        "location": faker.city(),
        "floor_number": random.randint(0, 30),
        "transaction_id": txn_fk,
        "customer_id": cust_fk,
        "portfolio_id": choose_or_none(list(range(1, NUM_PORTFOLIOS+1)), 0.67),
        "monthly_rent": round(np.random.uniform(4_000, 80_000), 2) if random.random()<0.35 else None
    })
df_Unit = pd.DataFrame(units)

# Link Project.unit_id randomly to existing Units (schema has no Unit.project_id)
any_unit_ids = df_Unit["unit_id"].tolist()
df_Project["unit_id"] = [random.choice(any_unit_ids) if any_unit_ids else None for _ in df_Project.index]

# =========================
# 9) Owner
# =========================
df_Owner = pd.DataFrame([{
    "owner_id": i,
    "first_name": faker.first_name(),
    "last_name": faker.last_name(),
    "address": faker.address().replace("\n", ", "),
    "phone": gen_unique_phone(),
    "email": gen_unique_email(),
    "type": random.choice(["Individual","Company"]),
    "registration_date": faker.date_between(start_date="-6y", end_date="today"),
    "unit_id": choose_or_none(any_unit_ids, 0.85)  # mostly NULL
} for i in range(1, NUM_OWNERS+1)])

# =========================
# 10) Owner_Ship
# =========================
owner_ship = []
pair = set()
for owner_id in df_Owner["owner_id"]:
    k = random.choice([0,1,1,2,3])
    if k:
        for u in random.sample(any_unit_ids, k=k):
            if (owner_id, u) not in pair:
                owner_ship.append({"Owner_id": owner_id, "Unit_id": u})
                pair.add((owner_id, u))
df_Owner_Ship = pd.DataFrame(owner_ship)

# =========================
# 11) Investor
# =========================
df_Investor = pd.DataFrame([{
    "investor_id": i,
    "investor_name": faker.name() if random.random()<0.6 else faker.company(),
    "phone": gen_unique_phone(),
    "email": gen_unique_email(),
    "company_name": faker.company() if random.random()<0.5 else None,
    "portfolio_id": choose_or_none(list(range(1, NUM_PORTFOLIOS+1)), 0.2)
} for i in range(1, NUM_INVESTORS+1)])

# =========================
# 12) Contractor
# =========================
contractors = []
for i in range(1, NUM_CONTRACTORS+1):
    s = faker.date_between(start_date="-5y", end_date="-2y")
    e = s + pd.Timedelta(days=30 * random.randint(30, 200))
    contractors.append({
        "contractor_id": i,
        "contractor_name": faker.company(),
        "phone": gen_unique_phone(),
        "email": gen_unique_email(company=True),
        "project_cost": round(np.random.uniform(100_000, 8_000_000), 2),
        "start_date": s,
        "end_date": e,
        "project_count": random.randint(1, 40),
        "payment_terms": random.choice(["30 days","60 days","Advance","Milestone"]),
        "project_id": random.randint(1, NUM_PROJECTS)
    })
df_Contractor = pd.DataFrame(contractors)

# =========================
# VALIDATIONS (mirror DDL)
# =========================
def assert_unique(df, cols, name):
    if df.duplicated(subset=cols).any():
        dups = df[df.duplicated(subset=cols, keep=False)].sort_values(cols)
        raise ValueError(f"[DUPLICATES] {name}: duplicate values in {cols}\n{dups.head(10)}")

def assert_fk(child_df, child_col, parent_df, parent_col, name):
    if child_df is None or len(child_df)==0: return
    child_vals = set(pd.to_numeric(child_df[child_col].dropna(), errors="coerce").dropna().astype(int))
    parent_vals = set(pd.to_numeric(parent_df[parent_col], errors="coerce").dropna().astype(int))
    missing = child_vals - parent_vals
    if missing:
        raise ValueError(f"[FK VIOLATION] {name}: {child_col}->{parent_col} missing: {list(missing)[:10]} ...")

# PKs
assert_unique(df_Segment, ["segment_id"], "Segment")
assert_unique(df_Portfolio, ["portfolio_id"], "Portfolio")
assert_unique(df_Payment_Method, ["payment_method_number"], "Payment_Method")
assert_unique(df_Project, ["project_id"], "Project")
assert_unique(df_Employee, ["Employee_id"], "Employee")
assert_unique(df_Customer, ["customer_id"], "Customer")
assert_unique(df_Transactions, ["transaction_id"], "Transactions")
assert_unique(df_Unit, ["unit_id"], "Unit")
assert_unique(df_Owner, ["owner_id"], "Owner")
assert_unique(df_Owner_Ship, ["Owner_id","Unit_id"], "Owner_Ship")
assert_unique(df_Investor, ["investor_id"], "Investor")
assert_unique(df_Contractor, ["contractor_id"], "Contractor")

# Business constraints
assert_unique(df_Customer, ["phone"], "Customer.phone UNIQUE")
if not set(df_Transactions["payment_status"]).issubset(set(tx_payment_status)):
    raise ValueError("[CHECK] Transactions.payment_status has values outside ('Paid','Pending','Cancelled')")

# FKs
assert_fk(df_Employee.dropna(subset=["project_id"]), "project_id", df_Project, "project_id", "Employee.project_id")
assert_fk(df_Customer.dropna(subset=["segment_id"]), "segment_id", df_Segment, "segment_id", "Customer.segment_id")
assert_fk(df_Customer.dropna(subset=["Employee_id"]), "Employee_id", df_Employee, "Employee_id", "Customer.Employee_id")
assert_fk(df_Transactions.dropna(subset=["customer_id"]), "customer_id", df_Customer, "customer_id", "Transactions.customer_id")
assert_fk(df_Transactions.dropna(subset=["payment_method_number"]), "payment_method_number", df_Payment_Method, "payment_method_number", "Transactions.payment_method_number")
assert_fk(df_Transactions.dropna(subset=["employee_id"]), "employee_id", df_Employee, "Employee_id", "Transactions.employee_id")
assert_fk(df_Unit.dropna(subset=["transaction_id"]), "transaction_id", df_Transactions, "transaction_id", "Unit.transaction_id")
assert_fk(df_Unit.dropna(subset=["customer_id"]), "customer_id", df_Customer, "customer_id", "Unit.customer_id")
assert_fk(df_Unit.dropna(subset=["portfolio_id"]), "portfolio_id", df_Portfolio, "portfolio_id", "Unit.portfolio_id")
assert_fk(df_Owner.dropna(subset=["unit_id"]), "unit_id", df_Unit, "unit_id", "Owner.unit_id")
assert_fk(df_Owner_Ship, "Owner_id", df_Owner, "owner_id", "Owner_Ship.Owner_id")
assert_fk(df_Owner_Ship, "Unit_id", df_Unit, "unit_id", "Owner_Ship.Unit_id")
assert_fk(df_Investor.dropna(subset=["portfolio_id"]), "portfolio_id", df_Portfolio, "portfolio_id", "Investor.portfolio_id")
assert_fk(df_Contractor.dropna(subset=["project_id"]), "project_id", df_Project, "project_id", "Contractor.project_id")

# Consistency: Units with txn must be Sale/Rent, and if Unit.customer_id set then == Transaction.customer_id
if not df_Unit["transaction_id"].isna().all():
    merged = df_Unit.merge(
        df_Transactions[["transaction_id","transaction_type","customer_id"]],
        on="transaction_id",
        how="left",
        suffixes=("_unit","_txn"),
    )
    linked = merged[merged["transaction_id"].notna()].copy()

    # 1) Only Sale/Rent for linked units
    bad_type = linked[~linked["transaction_type"].isin(["Sale","Rent"])]
    if not bad_type.empty:
        raise ValueError("[LOGIC] Unit linked to non-Sale/Rent:\n"
                         f"{bad_type[['unit_id','transaction_id','transaction_type']].head(10)}")

    # 2) Customer alignment (robust to suffix naming)
    unit_cust_col = "customer_id_unit" if "customer_id_unit" in linked.columns else \
                    ("customer_id_x" if "customer_id_x" in linked.columns else "customer_id")
    txn_cust_col  = "customer_id_txn"  if "customer_id_txn"  in linked.columns else \
                    ("customer_id_y" if "customer_id_y" in linked.columns else "customer_id")

    both = linked[unit_cust_col].notna() & linked[txn_cust_col].notna()
    bad_cust = linked[both & (linked[unit_cust_col].astype(int) != linked[txn_cust_col].astype(int))]
    if not bad_cust.empty:
        raise ValueError("[LOGIC] Unit.customer_id != Transaction.customer_id:\n"
                         f"{bad_cust[['unit_id','transaction_id', unit_cust_col, txn_cust_col]].head(10)}")

# =========================
# CLEAN & EXPORT (safe CSVs)
# =========================
tables = {
    "Segment":        df_Segment,
    "Portfolio":      df_Portfolio,
    "Payment_Method": df_Payment_Method,
    "Project":        df_Project,
    "Employee":       df_Employee,
    "Customer":       df_Customer,
    "Transactions":   df_Transactions,
    "Unit":           df_Unit,
    "Owner":          df_Owner,
    "Owner_Ship":     df_Owner_Ship,
    "Investor":       df_Investor,
    "Contractor":     df_Contractor,
}
tables = {k: clean_text_cols(v) for k,v in tables.items()}

for name, df in tables.items():
    df.to_csv(
        os.path.join(OUTPUT_PATH, f"{name}.csv"),
        index=False,
        quoting=csv.QUOTE_ALL,
        escapechar="\\",
        encoding="utf-8-sig"
    )

print("=== Data Generation Complete ===")
for n, d in tables.items():
    print(f"{n:14} rows={len(d):6} | null_avg={d.isna().mean().mean():.3f}")
print(f"All CSVs written to '{OUTPUT_PATH}'")


=== Data Generation Complete ===
Segment        rows=    60 | null_avg=0.000
Portfolio      rows=   400 | null_avg=0.000
Payment_Method rows=   600 | null_avg=0.000
Project        rows=   180 | null_avg=0.000
Employee       rows=   400 | null_avg=0.052
Customer       rows=  8000 | null_avg=0.013
Transactions   rows=  7000 | null_avg=0.016
Unit           rows=  9000 | null_avg=0.189
Owner          rows=  4500 | null_avg=0.001
Owner_Ship     rows=  6257 | null_avg=0.000
Investor       rows=   600 | null_avg=0.009
Contractor     rows=   300 | null_avg=0.000
All CSVs written to 'reocdb_faker'
